# Segment 4 Lab 1

## Introduction to this segment's labs

# Introducing: The Price Is Right

We are going to build a multi-agent framework that will:
1. Pull RSS feeds from the web and interpret them, looking for promising deals (multi-shot prompting, structured outputs)
2. Use RAG and a Frontier Model to estimate the price of a product, using a knowledge base
3. Use a QLoRA fine tuned LLM to estimate the price
4. Send a push notification when it finds a deal that's worth significantly less than it estimates
5. A planning agent will orchestrate this

### We'll be going through this fast - the idea is for you to come back and try it yourself!


## First: Let's look at the Agent code for interpreting RSS feeds using multi-shot prompting

In [1]:
# Some imports

from dotenv import load_dotenv
from agents.deals import ScrapedDeal
from openai import OpenAI
import logging

In [2]:
# Set up our env variables as before

load_dotenv(override=True)
openai = OpenAI()
MODEL = "gpt-4o-mini"

In [3]:
# I wrote this code to fetch RSS feeds

scraped_deals = ScrapedDeal.fetch(show_progress=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:08<00:00,  2.68s/it]


In [4]:
print(scraped_deals[3].describe())

Title: Dell Speakerphone for $74 + free shipping
Details: That's a savings of $6 and the first discount we've seen. Buy Now at Dell Technologies
Features: "The world’s most intelligent Microsoft Teams-certified speakerphone" Volume, microphone on/mute switch, status indicator, answer/end, power/streaming LED indicator, reject call, microphone volume, Microsoft Teams button, answer call AI Noise Cancellation filters out background noise in your environment True stereo output with left and right speakers  Full Duplex Audio allows multiple participants to speak simultaneously, and echo-cancellation eliminates interruptions USB connectivity Model: S
URL: https://www.dealnews.com/products/Dell/Dell-Speakerphone/487582.html?iref=rss-c142


In [5]:
SYSTEM_PROMPT = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
    
{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [6]:
USER_PROMPT_PREFIX = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
    
Deals:
    
"""

USER_PROMPT_SUFFIX = """\n\nStrictly respond in JSON and include exactly 5 deals, no more.
Your response must follow this format:

{"deals": [
    {
        "product_description": "Your summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    {
        "product_description": "Your summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 210.30,
        "url": "the url as provided"
    },
    ...
]}
"""

In [7]:
# this makes a suitable user prompt given scraped deals

def make_user_prompt(scraped):
    user_prompt = USER_PROMPT_PREFIX
    user_prompt += '\n\n'.join([scrape.describe() for scrape in scraped])
    user_prompt += USER_PROMPT_SUFFIX
    return user_prompt

In [8]:
user_prompt = make_user_prompt(scraped_deals)
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
    
Deals:
    
Title: TYLT 10W Qi Wireless Charging Tray for $15 + free shipping
Details: That's a $5 low. Buy Now at eBay
Features: 
URL: https://www.dealnews.com/TYLT-10-W-Qi-Wireless-Charging-Tray-for-15-free-shipping/21722245.html?iref=rss-c142

Title: Mint Mobile 3-Month Unlimited Plan SIM Kit for $51.99 + free shipping
Details: Most merchants charge $

In [9]:
# Create the user prompt that reflects the deals we have scraped

user_prompt = make_user_prompt(scraped_deals)

# Call OpenAI using their Python client. Request a response in JSON and stream it back.

stream = openai.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt}
  ],
    max_tokens=1000,
    stream=True,
    response_format={"type": "json_object"}
)

response = ""
for chunk in stream:
    chunk_text = chunk.choices[0].delta.content or ''
    response += chunk_text
    print(chunk_text, end="", flush=True)

{
    "deals": [
        {
            "product_description": "The Dell Speakerphone is designed for seamless communication and collaboration, featuring Microsoft Teams certification for an enhanced conferencing experience. It includes controls for volume, mute, and call answering, along with a status indicator. The speakerphone boasts AI Noise Cancellation to filter out background sounds, ensuring clear audio for all participants. With USB connectivity and full duplex audio, it allows multiple speakers to communicate effectively without interruptions.",
            "price": 74,
            "url": "https://www.dealnews.com/products/Dell/Dell-Speakerphone/487582.html?iref=rss-c142"
        },
        {
            "product_description": "The Apple iPad Air with M3 chip is a high-performance tablet offering unmatched speed and efficiency for various tasks. Available in 11\" and 13\" models, it features a range of storage options from 128GB to 1TB and supports WiFi or WiFi + Cellular conn

In [10]:
import json
json.loads(response)

{'deals': [{'product_description': 'The Dell Speakerphone is designed for seamless communication and collaboration, featuring Microsoft Teams certification for an enhanced conferencing experience. It includes controls for volume, mute, and call answering, along with a status indicator. The speakerphone boasts AI Noise Cancellation to filter out background sounds, ensuring clear audio for all participants. With USB connectivity and full duplex audio, it allows multiple speakers to communicate effectively without interruptions.',
   'price': 74,
   'url': 'https://www.dealnews.com/products/Dell/Dell-Speakerphone/487582.html?iref=rss-c142'},
  {'product_description': 'The Apple iPad Air with M3 chip is a high-performance tablet offering unmatched speed and efficiency for various tasks. Available in 11" and 13" models, it features a range of storage options from 128GB to 1TB and supports WiFi or WiFi + Cellular connectivity. The iPad Air is ideally suited for professionals and creatives al

# Putting this into an "Agent"

I've packaged this code into a class called `ScannerAgent`

There are various Agent frameworks that add an abstraction layer, but in our case it's easy just to write the code directly.

In [11]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [12]:
from agents.scanner_agent import ScannerAgent

scanner = ScannerAgent()
print(scanner.scan())

INFO:root:[Scanner Agent] Scanner Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is ready
INFO:root:[Scanner Agent] Scanner Agent is about to fetch deals from RSS feed
INFO:root:[Scanner Agent] Scanner Agent received 15 deals not already scraped
INFO:root:[Scanner Agent] Scanner Agent is calling OpenAI using Structured Output
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Scanner Agent] Scanner Agent received 5 selected deals with price>0 from OpenAI


deals=[Deal(product_description='The TYLT 10W Qi Wireless Charging Tray is a sleek and convenient charging solution for compatible smartphones. It features a minimalist design that fits perfectly on any desk or nightstand. With its advanced Qi technology, it can deliver up to 10 watts of power, ensuring a fast charging experience for your device. Its compact size makes it highly portable, ideal for both home and office use, and it includes a free shipping option for added convenience.', price=15.0, url='https://www.dealnews.com/TYLT-10-W-Qi-Wireless-Charging-Tray-for-15-free-shipping/21722245.html?iref=rss-c142'), Deal(product_description='The Mint Mobile 3-Month Unlimited Plan SIM Kit offers a flexible and budget-friendly mobile service. This plan provides unlimited talk, text, and data access with 5G coverage, ensuring that you stay connected without the worry of overage charges. Compatible with a wide range of unlocked devices, this plan allows you to enjoy unlimited streaming, soci